# 모델 구축

1. AI-Hub 데이터로 감정분석 모델을 먼저 학습
2. 크롤링 등의 방법으로 관광 명소 리뷰로 **도메인 전이 학습**
3. 관광 리뷰와 관련된 키워드를 강화해 최종 모델 구축


2. 학습 데이터 -> AI-Hub 데이터 사용

# AI-Hub 데이터로 감정분석 모델 학습

In [ ]:
# Parameters
max_len = 512
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

## 1.1. 데이터 준비

### 1.1.1 데이터 로드
일단 하나만 불러옴

TODO : 모든 데이터 불러오기

In [ ]:
from pathlib import Path
import os
import json
import glob
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

proj_root = Path('.').resolve().parent
data_path = os.path.join(proj_root, 'data', 'json')

json_files = glob.glob(data_path + '/TL_SNS_*/*.json', recursive=True)

### 1.1.2. 데이터셋 클래스 생성

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# DataSet Class

relevant_aspects = ['가격', '서비스', '품질']

class ABSADataset(Dataset):
    def __init__(self, json_files, tokenizer, max_len):
        self.sentences = []
        self.labels = []
        self.tokenizer = tokenizer
        self.max_len = max_len

        for file in json_files:
            with open(file, 'r', encoding='utf-8') as f:
                data = json.load(f)
                for review in data:
                    sentence = review["RawText"]
                    for aspect in review["Aspects"]:
                        aspect_term = aspect["Aspect"]
                        if aspect_term in relevant_aspects:
                            sentiment = int(aspect["SentimentPolarity"]) + 1 # 0: Negative, 1: Neutral, 2: Positive
                            combined_input = f"{sentence} [SEP] {aspect_term}"
                            self.sentences.append(combined_input)
                            self.labels.append(sentiment)

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        combined_input = self.sentences[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            combined_input,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

### 1.2 데이터 전처리(Tokenizer)

BERT에는 Tokenizer를 통해 입력데이터를 토큰화하여 모델에 입력함.


BERT모델은 일반적으로 다음을 입력으로 받음
1. input_ids: 토큰화된 입력 데이터의 ID
2. attention_mask: 유효한 토큰과 패딩 토큰을 구분하는 마스크(1: 유효한 토큰, 0: 패딩 토큰)
3. token_type_ids: 문장이 하나인지, 두개인지 구분하는 ID(0: 첫번째 문장, 1: 두번째 문장, 기본값은 0)

tokenizer는 Huggingface transformer에서 제공하는 라이브러리중에서 사용함. : skt/kobert-base-v1

tokenizer
1. texts
2. truncation=True : 길이가 max_length를 넘어가면 잘라냄
3. padding=True: 길이가 부족한 경우 패딩 추가
4. max_length=512: 최대 길이 설정
5. return_tensors="pt": 파이토치 텐서로 반환
6. return_token_type_ids=True: 토큰 타입 아이디 반환

In [ ]:
from kobert_tokenizer import KoBERTTokenizer

tokenizer = KoBERTTokenizer.from_pretrained("skt/kobert-base-v1", sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})

train_dataset = ABSADataset(json_files, tokenizer, max_len=max_len)

## 2. 모델 학습

## 2.1 모델 로드
Pytorch에서 데이터를 효율적으로 처리하고 학습에 필요한 배치 단위로 데이터를 공급
데이터를 학습에 사용할 수 있는 형태로 정리, 배치 단위로 나누거나 섞는 작업을 자동화함

Batch
학습 중 한 번에 모델에 입력되는 데이터의 개수
모든 데이터를 한번에 처리하지 않고, 일정한 크기로 나눠서 처리

In [ ]:
# 모델 로드
import torch
from transformers import BertForSequenceClassification

device = torch.device("cuda:0")
model = BertForSequenceClassification.from_pretrained("skt/kobert-base-v1", num_labels=3).to(device)

In [ ]:
# DataLoader 생성
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

## 2.2 옵티마이저, 손실함수 정의
- 옵티마이저 : 모델의 파라미터를 업데이트 하는 알고리즘
    - 목표 : 손실함수 값을 최소화 하는 방향으로 모델 파라미터 조정
    - adamW 사용 Adam에서 가중치 감소를 추가하여 과적합을 방지함
- 손실함수 : 모델의 출력(예측값)과 실제 정답(레이블)간의 차이를 수치적으로 나타낸 것
    - 해당 값을 기반으로 옵티마이저가 가중치를 업데이트함
    - BERT에서는 기본으로 Pytorch의 CrossEntropyLoss사용

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

# Secheduler
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

## 2.3 학습 루프

In [ ]:
print(f"Total samples: {len(train_dataset)}")

In [ ]:
from tqdm import tqdm
from torch.amp import autocast, GradScaler

scaler = GradScaler()

# 모델 학습
model.train()

for epoch in range(num_epochs):
    epoch_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False)
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with autocast("cuda:0"):
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            epoch_loss += loss.item()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        progress_bar.set_postfix(loss=loss.item())

    avg_loss = epoch_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

    torch.save(model.state_dict(), f"model_epoch_{epoch + 1}.pth")

## 2.4 모델 저장

In [ ]:
model.save_pretrained("kobert-finetuned")
tokenizer.save_pretrained("kobert-finetuned")

In [ ]:
# 모델 평가 예시
model.eval()
with torch.no_grad():
    test_sentence = "이 제품은 디자인이 정말 멋져요"
    test_aspect = "디자인"
    combined_input = f"{test_sentence} [SEP] {test_aspect}"
    encoding = tokenizer.encode_plus(
        combined_input,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()

    sentiment_map = {0: "부정", 1: "중립", 2: "긍정"}
    print(f"Aspect '{test_aspect}'에 대한 감성 분석 결과: {sentiment_map[prediction]}")